In [1]:
import sys, os
sys.path.insert(0, '../')
sys.path.insert(0, '../python_src/')

import numpy as np
import scipy as sp

import homology
import chomology

In [2]:

# NV = 6
# NE = 5

# DIM = 2

# edgei = [0, 0, 0, 0, 0]
# edgej = [1, 2, 3, 4, 5]

# L = np.array([10.0, 10.0], float)

# vert_pos = np.zeros(DIM*NV, float)
# vert_pos[0:DIM] = [0, 0]
# vert_pos[DIM:2*DIM] = [-1, 0]
# vert_pos[2*DIM:3*DIM] = [1, 0]
# vert_pos[3*DIM:4*DIM] = [0, -1]
# vert_pos[4*DIM:5*DIM] = [0, 1]
# vert_pos[5*DIM:6*DIM] = [1, 1]


NV = 5
NE = 6

DIM = 2

edgei = [0, 0, 1, 2, 1, 3]
edgej = [1, 2, 3, 3, 4, 4]

L = np.array([10.0, 10.0], float)

vert_pos = np.zeros(DIM*NV, float)
vert_pos[0:DIM] = [0, 1]
vert_pos[DIM:2*DIM] = [1, 1]
vert_pos[2*DIM:3*DIM] = [0, 0]
vert_pos[3*DIM:4*DIM] = [1, 0]
vert_pos[4*DIM:5*DIM] = [2.5, 0.5]

print("pos", vert_pos)
    
graph = chomology.Graph(NV, NE, edgei, edgej)
graph.set_embedding(DIM, vert_pos, L)

corners = chomology.find_corners(graph)

print(corners)

corner_strains = np.array([2, 4, 5, 0, 1, 3, 6])

graph_comp = chomology.construct_graph_complex(graph, False)
corner_comp = chomology.construct_corner_complex(corners, graph_comp, DIM)

print("Graph Complex:")

for i in range(graph_comp.ncells):
    print(i, graph_comp.get_label(i), graph_comp.get_dim(i), graph_comp.get_facets(i), graph_comp.get_cofacets(i))

print("Corner Complex:")
    
for i in range(corner_comp.ncells):
    print(i, corner_comp.get_label(i), corner_comp.get_dim(i), corner_comp.get_facets(i), corner_comp.get_cofacets(i))
    
dual = True
    
filt = chomology.Filtration(corner_comp, corner_strains, dual)
chomology.perform_watershed_transform(filt, corner_comp, dual)
chomology.construct_filtration(filt, corner_comp, dual)

print(filt.get_filtration())
# for i in range(corner_comp.ncells):
#     print(filt.get_filtration_order(i))

rfilt = chomology.reduce_filtration(filt, corner_comp, graph_comp, dual)

print(rfilt.get_filtration())

pos [ 0.   1.   1.   1.   0.   0.   1.   0.   2.5  0.5]
[[0, 1, 2], [1, 3, 4], [1, 0, 3], [2, 0, 3], [3, 1, 4], [3, 1, 2], [4, 1, 3]]
Graph Complex:
0 0 0 [] [5, 6]
1 1 0 [] [5, 7, 9]
2 2 0 [] [6, 8]
3 3 0 [] [7, 8, 10]
4 4 0 [] [9, 10]
5 0 1 [0, 1] []
6 1 1 [0, 2] []
7 2 1 [1, 3] []
8 3 1 [2, 3] []
9 4 1 [1, 4] []
10 5 1 [3, 4] []
Corner Complex:
0 0 0 [] [5, 6, 13, 14]
1 1 0 [] [5, 7, 9, 11, 15, 16, 17]
2 2 0 [] [6, 8, 11, 16]
3 3 0 [] [7, 8, 10, 12, 13, 14, 17]
4 4 0 [] [9, 10, 12, 15]
5 5 1 [0, 1] [18, 20]
6 6 1 [0, 2] [18, 21]
7 7 1 [1, 3] [19, 20, 22, 23]
8 8 1 [2, 3] [21, 23]
9 9 1 [1, 4] [19, 24]
10 10 1 [3, 4] [22, 24]
11 -1 1 [2, 1] [18]
12 -1 1 [4, 3] [19]
13 -1 1 [3, 0] [20]
14 -1 1 [3, 0] [21]
15 -1 1 [4, 1] [22]
16 -1 1 [2, 1] [23]
17 -1 1 [3, 1] [24]
18 0 2 [11, 6, 5] []
19 1 2 [12, 9, 7] []
20 2 2 [7, 13, 5] []
21 3 2 [8, 14, 6] []
22 4 2 [10, 15, 7] []
23 5 2 [8, 7, 16] []
24 6 2 [10, 9, 17] []
[2, 0, 3, 14, 6, 8, 21, 4, 1, 15, 7, 10, 22, 11, 5, 18, 16, 23, 12, 9, 19, 